# 24_Großetranke

In [13]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/24_Großetranke.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Grosse Trünke WehrUP", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+90)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+90)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Grosse Trünke WehrUP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=400, y=260, x_units='screen', y_units='screen',
              text='NS=0.38',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=400, y=240, x_units='screen', y_units='screen',
              text='R²=0.70',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 8_Garzau

In [15]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/8_Garzau.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Garzau", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+90)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+90)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Garzau", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=400, y=260, x_units='screen', y_units='screen',
              text='NS= 0.56',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=400, y=240, x_units='screen', y_units='screen',
              text='R²= 0.63',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 9_Fredersdorf

In [17]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/9_Fredersdorf.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Fredersdorf", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+90)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+90)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Fredersdorf", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=400, y=260, x_units='screen', y_units='screen',
              text='NS= 0.31',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=400, y=240, x_units='screen', y_units='screen',
              text='R²= 0.37',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 11_Dahlwitzhoppergarten

In [21]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/11_DahlwitzHoppergarten.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Dahlwitz_Hoppegarten", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+100)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+100)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Dahlwitz_Hoppegarten", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.25',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.52',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 12_Hiedekrug

In [23]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/12_Heidekruge.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Heidekrug", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+50)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Heidekrug", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.59',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.74',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 14_Kienbaum Sportschule

In [24]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/14_Kienbaum_Sportschule_UP.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Kienbaum_Sportschule_UP", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+50)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Kienbaum_Sportschule_UP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.55',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.73',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 18_Grunheide2

In [27]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/18_Grunheide2.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Grünheide_2", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+30)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+30)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Grünheide_2", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.47',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.66',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_center"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 29_Briesen

In [31]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/29_Briesen.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Briesen", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+70)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+70)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Briesen", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.61',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.64',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 52_Friedland wugglemuhle

In [33]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/52_Friedland_wugglemuhle.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Friedland_Wuggelmühle", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+50)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Friedland_Wuggelmühle", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.61',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.73',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 56_Goyatz 2

In [38]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/56_Goyatz2.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Goyatz_2", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+45)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+45)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Goyatz_2", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.78',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.80',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 59_pieskow

In [39]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/59_Pieskow.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Pieskow", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+45)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+45)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Pieskow", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.65',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.73',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 60_Schulenwise

In [41]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/60_Schulenwise.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Schuhlen_Wiese", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+40)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+40)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Schuhlen_Wiese", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.62',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.72',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 61_Mollen

In [43]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/61_Mollen1.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Möllen_1", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+30)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+30)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Möllen_1", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.68',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.72',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 66_Doberburg

In [45]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/66_Doberburg.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Doberburg", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+30)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+30)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Doberburg", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.69',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.76',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 76_Eggersdorf

In [47]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/76_Eggersdorf2.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Eggersdorf_2", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+50)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Eggersdorf_2", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.51',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.54',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 78_Altandsberg

In [48]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/78_Altlandsberg_2_Walkmühle.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Altlandsberg_2_Walkmühle", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+50)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Altlandsberg_2_Walkmühle", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.13',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.33',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 79_Bruchmühle_WehrUP

In [49]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/79_Bruchmühle_WehrUP.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Bruchmühle_WehrUP", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+50)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Bruchmühle_WehrUP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.42',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.46',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 81_ Schoniche

In [50]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/81_Schüneiche.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Schüneiche", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+50)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Schüneiche", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.32',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.50',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 82_Lichtenow

In [53]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/82_Lichtenow.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Lichtenow", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+40)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+40)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Lichtenow", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.14',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.44',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 83_Woltersdorf

In [55]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/83_Woltersdorf Schleuse op.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Woltersdorf_Schleuse_OP", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+50)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Woltersdorf_Schleuse_OP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.38',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.57',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 85_Kienbaum Straßenbrucke

In [56]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/85_Kienbaum_Straßenbrucke.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Kienbaum_Straßenbrücke", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+50)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Kienbaum_Straßenbrücke", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.33',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.69',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 92_Hohenbinde

In [57]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/92_Hohenbinde.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Hohenbinde", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+50)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Hohenbinde", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= -0.15',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.52',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 93_Berckenbrucke2

In [58]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/93_Berckenbrucke2.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Berkenbrück_2", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+50)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Berkenbrück_2", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.32',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.44',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 94_Berckenbrucke1

In [59]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/93_Berckenbrucke2.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Berkenbrück_1", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+50)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Berkenbrück_1", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.44',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.52',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 96_Kersdorf

In [60]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/96_Kersdorfschleuse op.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Kersdorf_Schleuse_OP", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+50)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Kersdorf_Schleuse_OP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.60',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.72',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 98_Furstenwalde

In [62]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/98_Fuerstenwalde.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Fuerstenwalde_OP", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+50)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Fuerstenwalde_OP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.75',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.79',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 99_Beeskow

In [63]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/99_Beeskow.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="BeeskowUP_Spreeschleuse", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+50)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="BeeskowUP_Spreeschleuse", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.56',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.69',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 100_Oegeln

In [64]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/100_Oegeln.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Oegeln", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+50)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Oegeln", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.67',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.73',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 101_Friedland pANKZERBRUCKE

In [66]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/101_Friedland_Panzerbrucke.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Friedland_Panzerbrücke", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+50)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Friedland_Panzerbrücke", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.69',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.75',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 102_Pretschen

In [67]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/102_Pretschen.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Pretschen", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+50)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Pretschen", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.63',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.73',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 108_Ressen Stau UP

In [68]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/108_Ressen Stau Up.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Ressen_Stau_UP", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+50)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Ressen_Stau_UP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.65',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.75',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 109_Doberburgmuhleup

In [70]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/109_Doberburg_Muhle_UP.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Doberburg_Mühle_UP", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+50)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Doberburg_Mühle_UP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.57',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.74',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# 110_Lieberose

In [72]:
#MODIFICATION INTO ONE CSV FILE
###################################### FOR ET ONLY ##################################################################
# Import Bokeh modules
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/110_Lieberose_Wehr_OP.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Grosse Trünke WehrUP_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Lieberose_Wehr_OP", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(0, df_beeskow['observed'].max()+40)
p.y_range = Range1d(0, df_beeskow['simulated'].max()+40)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Lieberose_Wehr_OP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS= 0.71',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²= 0.75',text_align='center',
              text_font_size='12pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "gray"
p.legend.label_text_font_style="bold"
p.legend.label_text_color="black"
p.legend.label_text_font_size="10pt"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"
# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)



# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# Beeskow

In [37]:
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/Beeskow.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Beeskow_.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="BeeskowUP Spreeschleuse", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(-20, df_beeskow['observed'].max()+50)
p.y_range = Range1d(-20, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="BeeskowUP Spreeschleuse", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS=0.57',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²=0.80',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "bottom_center"
p.legend.orientation = "horizontal"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "white"
p.legend.label_text_font_style="bold"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"


#discharge_avg_line = Span(location=simulated_avg.mean(), dimension='width', line_dash='dashed', line_color='black')
#precipitation_avg_line = Span(location=precipitation_avg.mean(), dimension='width', line_dash='dashed', line_color='green')
#p.add_layout(discharge_avg_line, 'left')
#p.add_layout(precipitation_avg_line, 'right')


# Calculate the trend line equation for the observed data
observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
observed_y = np.array(df_beeskow["observed"])
observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
observed_trendline = observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1]

# Calculate the trend line equation for the simulated data
simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
simulated_y = np.array(df_beeskow["simulated"])
simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]

# Add the trend lines to the plot
p.line(x=observed_x, y=observed_trendline, line_width=2.5, line_dash="dashed",color="orange", legend_label="Observed Trendline")
p.line(x=simulated_x, y=simulated_trendline, line_width=2.5, color="brown", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)

# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# Hohenbinde

In [38]:
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/Hohenbinde_final.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Hohenbinde.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Hohenbinde", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(-20, df_beeskow['observed'].max()+50)
p.y_range = Range1d(-20, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Hohenbinde", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS=0.19',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²=0.51',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "bottom_center"
p.legend.orientation = "horizontal"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "white"
p.legend.label_text_font_style="bold"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"


#discharge_avg_line = Span(location=simulated_avg.mean(), dimension='width', line_dash='dashed', line_color='black')
#precipitation_avg_line = Span(location=precipitation_avg.mean(), dimension='width', line_dash='dashed', line_color='green')
#p.add_layout(discharge_avg_line, 'left')
#p.add_layout(precipitation_avg_line, 'right')


# Calculate the trend line equation for the observed data
observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
observed_y = np.array(df_beeskow["observed"])
observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
observed_trendline = observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1]

# Calculate the trend line equation for the simulated data
simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
simulated_y = np.array(df_beeskow["simulated"])
simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]

# Add the trend lines to the plot
p.line(x=observed_x, y=observed_trendline, line_width=2.5, line_dash="dashed",color="orange", legend_label="Observed Trendline")
p.line(x=simulated_x, y=simulated_trendline, line_width=2.5, color="brown", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)

# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# Ressen Stau UP

In [39]:
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/Ressen_stau_up.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Hohenbinde.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Ressen_Stau_UP", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(-20, df_beeskow['observed'].max()+50)
p.y_range = Range1d(-20, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Ressen_Stau_UP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS=0.71',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²=0.71',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "bottom_center"
p.legend.orientation = "horizontal"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "white"
p.legend.label_text_font_style="bold"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"


#discharge_avg_line = Span(location=simulated_avg.mean(), dimension='width', line_dash='dashed', line_color='black')
#precipitation_avg_line = Span(location=precipitation_avg.mean(), dimension='width', line_dash='dashed', line_color='green')
#p.add_layout(discharge_avg_line, 'left')
#p.add_layout(precipitation_avg_line, 'right')


# Calculate the trend line equation for the observed data
observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
observed_y = np.array(df_beeskow["observed"])
observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
observed_trendline = observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1]

# Calculate the trend line equation for the simulated data
simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
simulated_y = np.array(df_beeskow["simulated"])
simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]

# Add the trend lines to the plot
p.line(x=observed_x, y=observed_trendline, line_width=2.5, line_dash="dashed",color="orange", legend_label="Observed Trendline")
p.line(x=simulated_x, y=simulated_trendline, line_width=2.5, color="brown", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)

# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# Kienbaum Sportschule

In [40]:
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/Ressen_stau_up.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Kienbaum_Sportschule_UP.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Kienbaum_Sportschule_UP", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(-20, df_beeskow['observed'].max()+50)
p.y_range = Range1d(-20, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Kienbaum_Sportschule_UP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS=0.69',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²=0.77',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "bottom_center"
p.legend.orientation = "horizontal"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "white"
p.legend.label_text_font_style="bold"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"


#discharge_avg_line = Span(location=simulated_avg.mean(), dimension='width', line_dash='dashed', line_color='black')
#precipitation_avg_line = Span(location=precipitation_avg.mean(), dimension='width', line_dash='dashed', line_color='green')
#p.add_layout(discharge_avg_line, 'left')
#p.add_layout(precipitation_avg_line, 'right')


# Calculate the trend line equation for the observed data
observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
observed_y = np.array(df_beeskow["observed"])
observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
observed_trendline = observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1]

# Calculate the trend line equation for the simulated data
simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
simulated_y = np.array(df_beeskow["simulated"])
simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]

# Add the trend lines to the plot
p.line(x=observed_x, y=observed_trendline, line_width=2.5, line_dash="dashed",color="orange", legend_label="Observed Trendline")
p.line(x=simulated_x, y=simulated_trendline, line_width=2.5, color="brown", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)

# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# Frustenwalde

In [42]:
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/Frustenwaldeop.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Fuerstenwalde_OP.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Fuerstenwalde_OP", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(-20, df_beeskow['observed'].max()+50)
p.y_range = Range1d(-20, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Fuerstenwalde_OP", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS=0.69',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²=0.71',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "bottom_center"
p.legend.orientation = "horizontal"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "white"
p.legend.label_text_font_style="bold"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"


#discharge_avg_line = Span(location=simulated_avg.mean(), dimension='width', line_dash='dashed', line_color='black')
#precipitation_avg_line = Span(location=precipitation_avg.mean(), dimension='width', line_dash='dashed', line_color='green')
#p.add_layout(discharge_avg_line, 'left')
#p.add_layout(precipitation_avg_line, 'right')


# Calculate the trend line equation for the observed data
observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
observed_y = np.array(df_beeskow["observed"])
observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
observed_trendline = observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1]

# Calculate the trend line equation for the simulated data
simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
simulated_y = np.array(df_beeskow["simulated"])
simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]

# Add the trend lines to the plot
p.line(x=observed_x, y=observed_trendline, line_width=2.5, line_dash="dashed",color="orange", legend_label="Observed Trendline")
p.line(x=simulated_x, y=simulated_trendline, line_width=2.5, color="brown", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)

# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")

# Berkenbrucke2

In [41]:
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, HoverTool,Title,BoxAnnotation,Label
from bokeh.models import DatetimeTickFormatter
from bokeh.models import YearsTicker
from bokeh.models import DatetimeTicker 
from bokeh.models import BasicTicker
from datetime import datetime as dt
from datetime import timedelta
from bokeh.models import Legend
from bokeh.models import LabelSet
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span
import pandas as pd
from bokeh.io import export_png

# Read the observed and simulated data from csv files
df_beeskow = pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/ET for Bokeh Plotting/Berkenbrucke2.csv')
#df_precip =pd.read_csv('E:/TUB Thesis Folder/Bokeh Plotting/Precipitation_DWD.csv',parse_dates=["Date"])

# Convert the date columns to datetime format
df_beeskow["Date"] = pd.to_datetime(df_beeskow["Date"])
#df_precip["Date"] = pd.to_datetime(df_precip["Date"])


# Create a ColumnDataSource object for observed data
source_beeskow = ColumnDataSource(df_beeskow)
#source_precip = ColumnDataSource(df_precip)

# Define the output file name and format
output_file("Calibrated ET Berkenbrück_2.html")


##width=1200,height=500
# Create a figure object with title, axis labels and tools
p = figure(title="Berkenbrück_2", x_axis_label="Year", y_axis_label="ET (mm/M)",  x_axis_type="datetime",width=900,height=400, tools="pan,wheel_zoom,box_zoom,reset,save")


# get the start and end date
start_date = df_beeskow['Date'].min()
end_date = df_beeskow['Date'].max()

# Create a YearsTicker object with desired number of ticks as 10 (50 years range)
years_interval = 3
## for daily------>ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 365 / years_interval) + 1)
ticker = YearsTicker(desired_num_ticks=int((end_date - start_date).days / 366 / years_interval) + 3)

# Set the number of minor tick marks between major tick marks to 4
p.xaxis[0].ticker.num_minor_ticks = 4

# Set x-axis ticker to YearsTicker object
p.xaxis.ticker = ticker

# Set x-axis tick label format to "mm/dd/yyyy"
date_format = "%-m/%-d/%Y"
p.xaxis.formatter = DatetimeTickFormatter(years=date_format)


# plot observed and simulated data as separate glyphs
observed_glyph = p.line(x="Date", y="observed", source=source_beeskow, line_width=2, color="red", name="observed", legend_label="Observed")
simulated_glyph = p.line(x="Date", y="simulated", source=source_beeskow, line_width=2, color="blue", name="simulated", legend_label="Simulated")
p.y_range = Range1d(-20, df_beeskow['observed'].max()+50)
p.y_range = Range1d(-20, df_beeskow['simulated'].max()+50)


# create precipitation bar chart on right y-axis
#p.extra_y_ranges = {"24": Range1d(start=max(df_precip['24'])+20, end=0)}
#precip_glyph=p.vbar(x="Date", top="24", width=0.5, color='skyblue', source=source_precip, y_range_name="24", legend_label='Precipitation')
#p.add_layout(LinearAxis(y_range_name="24",axis_label="Precipitation (mm)", ticker=BasicTicker(), axis_line_color='orange'), 'right')


# add a hover tool to the figure to show the date and discharge values on mouseover
hover_observed = HoverTool(tooltips=[("Observed", "@observed{0.00}")], renderers=[observed_glyph])
hover_simulated = HoverTool(tooltips=[("Simulated", "@simulated{0.00}")], renderers=[simulated_glyph])
#hover_precip = HoverTool(tooltips=[("Precipitation", "@24{0.00000}")], renderers=[precip_glyph])
#p.add_tools(hover_observed, hover_simulated,hover_precip)
p.add_tools(hover_observed, hover_simulated)


# Add a rectangular border to the figure using the outline_line_width and outline_line_color arguments
p.outline_line_width = 1.5 # Set the width of the border line
p.outline_line_color = "brown" # Set the color of the border line

# Create a title for the plot
title = Title(text="Berkenbrück_2", align="center", text_font_size="18pt", text_font_style="bold")
p.title = title



##x=500, y=250
##x=500, y=230

# Create a label object for ns
label_ns = Label(x=80, y=260, x_units='screen', y_units='screen',
              text='NS=0.54',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_ns)

# Create a label object for r2
label_r2 = Label(x=80, y=240, x_units='screen', y_units='screen',
              text='R²=0.65',text_align='center',
              text_font_size='10pt', text_font_style='bold')
p.add_layout(label_r2)

# Create a label object for kge
#label_kge = Label(x=500, y=210, x_units='screen', y_units='screen',
#              text='KGE=0.91',text_align='center',
#              text_font_size='10pt', text_font_style='bold')
#p.add_layout(label_kge)


# Applying theme to the plot using curdoc().theme argument
from bokeh.io import curdoc # Import curdoc() function
curdoc().theme = "light_minimal" 



#Customization of the title and axis labels using text_font_size and text_font_style arguments
p.title.text_font_size = "18pt" 
p.title.text_font_style = "bold" 
p.xaxis.axis_label_text_font_size = "14pt" 
p.yaxis.axis_label_text_font_size = "14pt"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label_text_font_style="bold"


#### Customize the legend using location, orientation and background_fill_alpha arguments
p.legend.location = "bottom_center"
p.legend.orientation = "horizontal"
p.legend.background_fill_alpha = 0.3
p.legend.background_fill_color = "white"
p.legend.label_text_font_style="bold"
#p.legend.label_text_font_size="5"
#p.legend.label_width = 5
#p.legend.border_line_alpha=0.9
#p.legend.border_line_color = "navy"


#discharge_avg_line = Span(location=simulated_avg.mean(), dimension='width', line_dash='dashed', line_color='black')
#precipitation_avg_line = Span(location=precipitation_avg.mean(), dimension='width', line_dash='dashed', line_color='green')
#p.add_layout(discharge_avg_line, 'left')
#p.add_layout(precipitation_avg_line, 'right')


# Calculate the trend line equation for the observed data
observed_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
observed_y = np.array(df_beeskow["observed"])
observed_coef = np.polyfit(observed_x.astype(np.int64) // 10**9, observed_y, 1)
observed_trendline = observed_coef[0] * observed_x.astype(np.int64) // 10**9 + observed_coef[1]

# Calculate the trend line equation for the simulated data
simulated_x = np.array(df_beeskow["Date"], dtype=np.datetime64)
simulated_y = np.array(df_beeskow["simulated"])
simulated_coef = np.polyfit(simulated_x.astype(np.int64) // 10**9, simulated_y, 1)
simulated_trendline = simulated_coef[0] * simulated_x.astype(np.int64) // 10**9 + simulated_coef[1]

# Add the trend lines to the plot
p.line(x=observed_x, y=observed_trendline, line_width=2.5, line_dash="dashed",color="orange", legend_label="Observed Trendline")
p.line(x=simulated_x, y=simulated_trendline, line_width=2.5, color="brown", line_dash="dashed", legend_label="Simulated Trendline")

# remove grid lines in the plot area
p.grid.grid_line_color = 'white'


window_size = 2  # Define the window size
df_beeskow['simulated_std'] = df_beeskow['simulated'].rolling(window=window_size).std()

# Calculate upper and lower limits for 95% prediction uncertainty (ppu) band
df_beeskow["upper_limit"] = df_beeskow["simulated"] + 1.96 * df_beeskow["simulated_std"]
df_beeskow["lower_limit"] = df_beeskow["simulated"] - 1.96 * df_beeskow["simulated_std"]

# Create a ColumnDataSource object for the ppu band
source_ppu = ColumnDataSource(df_beeskow)

# Add the ppu line to the figure
ppu_line = p.line(x="Date", y="simulated", source=source_ppu, line_width=2)

# Add a shaded area for the standard deviation
ppu_band = p.varea(x="Date", y1="lower_limit", y2="upper_limit", source=source_ppu, color="green", alpha=0.2)

# Add a hover tool to the ppu line
hover_ppu = HoverTool(tooltips=[("PPU", "@simulated{0.00}")], renderers=[ppu_line])

# Add the hover tool to the figure
p.add_tools(hover_ppu)

# Show the figure in a browser
show(p)
#save(p)

#export_png(p, filename="plot.png")